In [2]:
from mlband.imports import *
from mlband.data import *
from mlband.model import *
from mlband.args import *


In [3]:
cifpath = Path('data/mp/')
dataset = CIFData(cifpath, radius=15)
train_loader, val_loader, test_loader = get_train_val_test_loader(
    dataset=dataset,
    collate_fn=collate_pool,
    batch_size=args.batch_size,
    train_ratio=args.train_ratio,
    num_workers=args.workers,
    val_ratio=args.val_ratio,
    test_ratio=args.test_ratio,
    pin_memory=args.cuda,
    train_size=args.train_size,
    val_size=args.val_size,
    test_size=args.test_size,
    return_test=True)

In [4]:
sample_data_list = [dataset[i] for i in
                    random.sample(range(len(dataset)), min(500, len(dataset)))]

/Users/ali/mambaforge/envs/cgcnn/lib/python3.11/site-packages/pymatgen/io/cif.py:1186: UserWarning: The default value of primitive was changed from True to False in https://github.com/materialsproject/pymatgen/pull/3419. CifParser now returns the cell in the CIF file as is. If you want the primitive cell, please set primitive=True explicitly.
  warnings.warn(
/Users/ali/mambaforge/envs/cgcnn/lib/python3.11/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 20 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/ali/mambaforge/envs/cgcnn/lib/python3.11/site-packages/pymatgen/io/cif.py:1219: UserWarning: Issues encountered while parsing CIF: 28 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
/Users/ali/mam

In [5]:
from mlband.model import Normalizer

In [7]:
_, sample_target, _ = collate_pool(sample_data_list)
normalizer = Normalizer(sample_target)

In [21]:
orig_atom_fea_len, nbr_fea_len

(92, 76)

In [19]:
structures, _, _ = dataset[0]
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]
model = CrystalGraphConvNet(orig_atom_fea_len, nbr_fea_len,
                            atom_fea_len=args.atom_fea_len,
                            n_conv=args.n_conv,
                            h_fea_len=args.h_fea_len,
                            n_h=args.n_h,
                            classification=True if args.task ==
                                                    'classification' else False)

In [11]:
model

CrystalGraphConvNet(
  (embedding): Linear(in_features=92, out_features=64, bias=True)
  (convs): ModuleList(
    (0-2): 3 x ConvLayer(
      (fc_full): Linear(in_features=204, out_features=128, bias=True)
      (sigmoid): Sigmoid()
      (softplus1): Softplus(beta=1, threshold=20)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (softplus2): Softplus(beta=1, threshold=20)
    )
  )
  (conv_to_fc): Linear(in_features=64, out_features=128, bias=True)
  (conv_to_fc_softplus): Softplus(beta=1, threshold=20)
  (fc_out): Linear(in_features=128, out_features=1, bias=True)
)